[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lweitkamp/GANs-JAX/blob/main/1_GANs.ipynb)


# Generative Adversarial Networks (GANs)

This notebook is based primarily on two papers. The first is the <a href="https://papers.nips.cc/paper/2014/file/5ca3e9b122f61f8f06494c97b1afccf3-Paper.pdf">original paper</a> on GANs (not counting Schmidhuber), the <a href="https://arxiv.org/abs/1511.06434">second paper</a> outlines improvements for training GANs and investigates the features learned by the generator for downstream tasks.


In the GAN setup, we want to implicity model a data distribution $p_{\text{data}}(x)$. The generator (G) is in charge of modelling $p_\text{data}$ through $p_g(x)$, and the discriminator (D) is in charge discerning data samples from the two distributions by a classification: 1 if the data is sampled from $p_{\text{data}}$ and 0 otherwise.


Formally this can be captured in a value function $V(D, G)$, where the generator and discriminator play the following minimax game:

$$
\large{
\underset{G}{\min} \underset{D}{\max} V(D, G) = \mathbb{E}_{x \sim p_{\text{data}}(x)}[\log D(x)] + \mathbb{E}_{z \sim p_z(z)}[\log (1 - D(G(z)))]
}
$$


GAN training is notoriously difficult, the following problems are typically prevalent:

1. Mode Collapse: The generator learns a specific subset of the data distribution (a mode), and the discriminator learns to classify these as false. The generator loops to a new mode and the problem continues.
2. <a href="https://arxiv.org/pdf/1701.04862.pdf">Vanishing Gradients</a>: If the discriminator converges too quickly, the signal back to the generator can be significantly small such that training is impossible for the generator. 
3. Random issues with convergence; both papers outline ways to combat these, for example label smoothing, batch-norm, tweaking the loss function for the generator, etc.

In [1]:
%%capture
!pip install --upgrade -q flax

In [2]:
# Run it on a TPU (if possible)!
import jax.tools.colab_tpu
try:
  jax.tools.colab_tpu.setup_tpu()
except:
  pass

In [3]:
import os
from functools import partial

import jax
import jax.numpy as jnp

from typing import Any, Tuple


PRNGKey = jnp.ndarray


num_devices = jax.device_count()

args = {
    'z_dim': 64,
    'seed': 41,
}

# Set the parallel batch size.
args['batch_size'] = num_devices * 256 if num_devices > 1 else 256
args['epochs'] = 100
args['batch_size_p'] = args['batch_size'] //  num_devices

args['true_label'] = jnp.ones((args['batch_size_p'], 1), dtype=jnp.int32)
args['false_label'] = jnp.zeros((args['batch_size_p'], 1), dtype=jnp.int32)

os.makedirs(os.path.join("results", "1_GAN"), exist_ok=True)

## Loading MNIST

In [4]:
import tensorflow_datasets as tfds
import tensorflow as tf


def set_range(batch):
  batch = tf.image.convert_image_dtype(batch['image'], tf.float32)
  batch = (batch - 0.5) / 0.5  # tanh range is -1, 1
  return batch
 

mnist_data = tfds.load("mnist")['train']
batches_in_epoch = len(mnist_data) // args['batch_size']
 
data_gen = iter(tfds.as_numpy(
      mnist_data
        .map(set_range)
        .cache()
        .shuffle(len(mnist_data), seed=args['seed'])
        .repeat()
        .batch(args['batch_size'])
 ))

Dl Completed...:   0%|          | 0/4 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to ~/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


## Flax Linen Architecture

In [5]:
from jax.nn.initializers import normal as normal_init
from flax.training import train_state
from flax import linen as nn


class TrainState(train_state.TrainState):
  batch_stats: Any


class Generator(nn.Module):
  features: int = 64
  dtype: type = jnp.float32

  @nn.compact
  def __call__(self, z: jnp.ndarray, train: bool = True):
    conv_transpose = partial(nn.ConvTranspose, padding='VALID',
                             kernel_init=normal_init(0.02), dtype=self.dtype)
    batch_norm = partial(nn.BatchNorm, use_running_average=not train, axis=-1, 
                         scale_init=normal_init(0.02), dtype=self.dtype)

    z = z.reshape((args['batch_size_p'], 1, 1, args['z_dim']))
    x = conv_transpose(self.features*4, kernel_size=[3, 3], strides=[2, 2])(z)
    x = batch_norm()(x)
    x = nn.relu(x)
    x = conv_transpose(self.features*4, kernel_size=[4, 4], strides=[1, 1])(x)
    x = batch_norm()(x)
    x = nn.relu(x)
    x = conv_transpose(self.features*2, kernel_size=[3, 3], strides=[2, 2])(x)
    x = batch_norm()(x)
    x = nn.relu(x)
    x = conv_transpose(1, [4, 4], [2, 2])(x)
    x = jnp.tanh(x)
    return x


class Discriminator(nn.Module):
  features: int = 64
  dtype: Any = jnp.float32

  @nn.compact
  def __call__(self, x: jnp.ndarray, train: bool = True):
    conv = partial(nn.Conv, kernel_size=[4, 4], strides=[2, 2], padding='VALID',
                   kernel_init=normal_init(0.02), dtype=self.dtype)
    batch_norm = partial(nn.BatchNorm, use_running_average=not train, axis=-1,
                         scale_init=normal_init(0.02), dtype=self.dtype)
        
    x = conv(self.features)(x)
    x = batch_norm()(x)
    x = nn.leaky_relu(x, 0.2)
    x = conv(self.features*2)(x)
    x = batch_norm()(x)
    x = nn.leaky_relu(x, 0.2)
    x = conv(1)(x)
    x = x.reshape((args['batch_size_p'], -1))
    return x

## GAN Loss Function

In [6]:
import optax


@partial(jax.pmap, axis_name='num_devices')
def generator_step(generator_state: TrainState,
                   discriminator_state: TrainState,
                   key: PRNGKey):
  r"""The generator is updated by generating data and letting the discriminator
  critique it. It's loss goes down if the discriminator wrongly predicts it to
  to be real data."""
  input_noise = jax.random.normal(key, (args['batch_size_p'], args['z_dim']))

  def loss_fn(params):
    generated_data, mutables = generator_state.apply_fn(
        {'params': params, 'batch_stats': generator_state.batch_stats},
        input_noise, mutable=['batch_stats'])
    
    logits, _ = discriminator_state.apply_fn(
        {'params': discriminator_state.params, 
         'batch_stats': discriminator_state.batch_stats},
         generated_data, mutable=['batch_stats'])
    
    loss = -jnp.mean(jnp.log(nn.sigmoid(logits)))
    return loss, mutables
  
  # Generate data with the Generator, critique it with the Discriminator.
  grad_fn = jax.value_and_grad(loss_fn, has_aux=True)
  (loss, mutables), grads = grad_fn(generator_state.params)

  # Average across the devices.
  grads = jax.lax.pmean(grads, axis_name='num_devices')
  loss = jax.lax.pmean(loss, axis_name='num_devices')

  # Update the Generator through gradient descent.
  new_generator_state = generator_state.apply_gradients(
      grads=grads, batch_stats=mutables['batch_stats'])
  
  return new_generator_state, loss


@partial(jax.pmap, axis_name='num_devices')
def discriminator_step(generator_state: TrainState,
                       discriminator_state: TrainState,
                       real_data: jnp.ndarray,
                       key: PRNGKey):
  r"""The discriminator is updated by critiquing both real and generated data,
  It's loss goes down as it predicts correctly if images are real or generated.
  """
  input_noise = jax.random.normal(key, (args['batch_size_p'], args['z_dim']))

  generated_data, _ = generator_state.apply_fn(
        {'params': generator_state.params, 
         'batch_stats': generator_state.batch_stats},
         input_noise, mutable=['batch_stats'])
  
  def loss_fn(params):
    logits_real, mutables = discriminator_state.apply_fn(
        {'params': params, 'batch_stats': discriminator_state.batch_stats},
        real_data, mutable=['batch_stats'])
        
    logits_generated, mutables = discriminator_state.apply_fn(
        {'params': params, 'batch_stats': mutables['batch_stats']},
        generated_data, mutable=['batch_stats'])
    
    real_loss = optax.sigmoid_binary_cross_entropy(
        logits_real, args['true_label']).mean()
    generated_loss = optax.sigmoid_binary_cross_entropy(
        logits_generated, args['false_label']).mean()
    
    loss = (real_loss + generated_loss) / 2

    return loss, mutables

  # Critique real and generated data with the Discriminator.
  grad_fn = jax.value_and_grad(loss_fn, has_aux=True)
  (loss, mutables), grads = grad_fn(discriminator_state.params)

  # Average cross the devices.
  grads = jax.lax.pmean(grads, axis_name='num_devices')
  loss = jax.lax.pmean(loss, axis_name='num_devices')

  # Update the discriminator through gradient descent.
  new_discriminator_state = discriminator_state.apply_gradients(
      grads=grads, batch_stats=mutables['batch_stats'])
  
  return new_discriminator_state, loss

## Helper Functions

In [7]:
@partial(jax.pmap, static_broadcasted_argnums=(1, 2))
def create_state(rng, model_cls, input_shape): 
  r"""Create the training state given a model class. """ 
  model = model_cls()

  tx = optax.adam(0.0002, b1=0.5, b2=0.999)
  variables = model.init(rng, jnp.ones(input_shape))

  state = TrainState.create(apply_fn=model.apply, tx=tx, 
      params=variables['params'], batch_stats=variables['batch_stats'])
  
  return state


@jax.pmap
def sample_from_generator(generator_state, input_noise):
  """Sample from the generator in evaluation mode."""
  generated_data = generator_state.apply_fn(
      {'params': generator_state.params,
       'batch_stats': generator_state.batch_stats},
      input_noise, train=False, mutable=False)
  return generated_data

## Training

In [8]:
from flax.training.common_utils import shard, shard_prng_key


key = jax.random.PRNGKey(seed=args['seed'])
key_generator, key_discriminator, key = jax.random.split(key, 3)
key_generator = shard_prng_key(key_generator)
key_discriminator = shard_prng_key(key_discriminator)

discriminator_state = create_state(key_discriminator, Discriminator, 
    (args['batch_size_p'], *next(data_gen).shape[1:]))

generator_state = create_state(key_generator, Generator, 
    (args['batch_size_p'], args['z_dim']))

generator_input = jax.random.normal(key, (args['batch_size'], args['z_dim']))
generator_input = shard(generator_input)

In [ ]:
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm


for epoch in tqdm(range(1, args['epochs'] + 1), desc="Epoch...",
                  position=0, leave=True):
  
  with tqdm(total=batches_in_epoch, desc="Training...",
            leave=False) as progress_bar_train:
    for batch in range(batches_in_epoch):

      # Generate RNG keys for generator and discriminator.
      key, key_generator, key_discriminator = jax.random.split(key, 3)
      key_generator = shard_prng_key(key_generator)
      key_discriminator = shard_prng_key(key_discriminator)

      # Shard the data to possible devices.
      batch_data = shard(next(data_gen))

      # Take a step with the generator.
      generator_state, generator_loss = generator_step(generator_state, 
          discriminator_state, key_generator)

      # Take a step with the discriminator.
      discriminator_state, discriminator_loss = discriminator_step(
          generator_state, discriminator_state, batch_data, key_discriminator)

      progress_bar_train.update(1)
    
    metrics = jax.device_get([generator_loss[0], discriminator_loss[1]])

    message = f"Epoch: {epoch: <2} | "
    message += f"Generator loss: {metrics[0]:.3f} | "
    message += f"Discriminator loss: {metrics[1]:.3f}"
    progress_bar_train.write(message)

  # Sample from the generator using the fixed input. We need to
  # reshape if we are working on multiple devices.
  sample = sample_from_generator(generator_state, generator_input)
  sample = sample.reshape((-1, 28, 28))

  # Next, plot the static samples, save the fig to disk.
  if epoch % 10 == 0:
    fig, axes = plt.subplots(nrows=10, ncols=10, figsize=(10, 10))
    for ax, image in zip(sum(axes.tolist(), []), sample):
      ax.imshow(image, cmap='gray')
      ax.set_axis_off()
    fig.savefig(f"results/1_GAN/GAN_epoch_{epoch}.png")
    plt.close(fig)

Epoch...:   0%|          | 0/100 [00:00<?, ?it/s]

Training...:   0%|          | 0/234 [00:00<?, ?it/s]

In [ ]:
import matplotlib.image as mpimg

# Load latest epoch results.
img = mpimg.imread(os.path.join('results', '1_GAN', f'GAN_epoch_{epoch}.png'))


fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(img, interpolation='nearest')
plt.axis('off')
plt.tight_layout()
plt.show()